In [1]:
import numpy as np
import pandas as pd

In [15]:
zip_ = pd.read_csv('zip_to_msa.csv',dtype='str').drop(columns=['Unnamed: 4'])

In [4]:
zip_.msa_name = zip_.msa_name.replace('nan', np.nan)
pop.msa_name = pop.msa_name.replace('nan', np.nan)

In [5]:
zip_.msa_name.dropna(inplace=True)
pop.msa_name.dropna(inplace=True)

In [6]:
def clean(x):
    return ('').join(((' ').join(x.split("-"))).split(","))

In [7]:
pop.msa_name = pop.msa_name.apply(lambda x: clean(x))

In [8]:
zip_.msa_name = zip_.msa_name.apply(lambda x: clean(x))

In [9]:
zip_.msa_name

0                 Boston Cambridge Quincy MA NH Metro Area
1                 Boston Cambridge Quincy MA NH Metro Area
2                 Boston Cambridge Quincy MA NH Metro Area
3                 Boston Cambridge Quincy MA NH Metro Area
4                 Boston Cambridge Quincy MA NH Metro Area
5                 Boston Cambridge Quincy MA NH Metro Area
6        New York Northern New Jersey Long Island NY NJ...
7        New York Northern New Jersey Long Island NY NJ...
8                                  PR NONMETROPOLITAN AREA
9            Aguadilla Isabela San Sebastian PR Metro Area
10           Aguadilla Isabela San Sebastian PR Metro Area
11           Aguadilla Isabela San Sebastian PR Metro Area
12           Aguadilla Isabela San Sebastian PR Metro Area
13                                 PR NONMETROPOLITAN AREA
14           Aguadilla Isabela San Sebastian PR Metro Area
15                                 PR NONMETROPOLITAN AREA
16                  San Juan Caguas Guaynabo PR Metro Ar

In [18]:
from fuzzywuzzy import fuzz
name = 'temp'
temp = 0
d = {}
for msa_zip in list(set(zip_.msa_name)):
    for msa_pop in list(set(pop.msa_name)):
        ratio = fuzz.token_sort_ratio(msa_zip, msa_pop)
        if ratio > temp:
            temp, name = ratio, msa_pop
    if temp >= 70:
        d[msa_zip] = name
        temp = 0

In [20]:
def try_fuzz(x):
    try:
        return d[x]
    except:
        return np.nan

In [21]:
zip_['msa_name_fuzz'] = zip_.msa_name.apply(lambda x: try_fuzz(x))

In [22]:
merged_msa = pd.merge(left=zip_ , right=pop, how='left', left_on='msa_name_fuzz', right_on='msa_name')

In [23]:
merged_msa.dropna(inplace=True)

In [24]:
merged_msa.columns

Index(['zip_code', 'state', 'msa_num', 'msa_name_x', 'msa_name_fuzz',
       'msa_name_y', 'pop_2018'],
      dtype='object')

In [30]:
df = merged_msa[['msa_name_fuzz','msa_name_x']]

In [20]:
merged_msa.to_csv('zipcode_population.csv')